In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('data/heart.csv')

data.info()

In [ ]:
def outlier(table, column: str):
    column_z = column + 'Z'
    table[column_z] = (table[column] - table[column].mean()) / table[column].std()

    index = data[ data[column_z] > 4].index
    data.drop(index, inplace = True)
    index = data[ data[column_z] < -4].index
    data.drop(index, inplace = True)

    data.drop(columns = column_z, inplace = True)
    return

In [ ]:
outlier(data, 'Age')
outlier(data, 'Oldpeak')
outlier(data, 'MaxHR')
outlier(data, 'RestBP')
data.describe()